## spark data 처리

1. 로컬 stock_min 정보를 읽어 warehouse구성
2. 구성 위치는 1차로 로컬 dw 하위로 stocklab db 를 구성
3. 이후 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import lit
 
db_path = '/content/drive/MyDrive/data-warehouse/stocklab_db'

spark = SparkSession.builder.appName('test_spark') \
  .config("spark.sql.warehouse.dir", db_path) \
  .enableHiveSupport() \
  .getOrCreate()